In [7]:
import os
import pandas as pd


def merge_datasets(directory):
    # Dictionary to hold pairs of filenames: {protein_name: {metric: [main_file, pareto_file]}}
    file_pairs = {}

    # List all CSV files in the directory
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            # Determine if the file is a pareto file
            is_pareto = "_pareto_" in filename
            # Extract the metric and protein name from the filename
            parts = filename.replace("_pareto", "").split("_")
            protein_name = parts[-1].split(".")[0]
            metric = "_".join(parts[:-1])

            # Initialize dictionary structure
            if protein_name not in file_pairs:
                file_pairs[protein_name] = {}
            if metric not in file_pairs[protein_name]:
                file_pairs[protein_name][metric] = [
                    None,
                    None,
                ]  # [main_file, pareto_file]

            # Assign filename to the correct slot
            if is_pareto:
                file_pairs[protein_name][metric][1] = filename
            else:
                file_pairs[protein_name][metric][0] = filename

    # Define the subdirectory
    subdirectory = "concat"

    # Merge datasets
    for protein, metrics in file_pairs.items():
      for metric, files in metrics.items():
        main_file, pareto_file = files
        if main_file and pareto_file:  # Check if both files are available
          df_main = pd.read_csv(os.path.join(directory, main_file))
          df_pareto = pd.read_csv(os.path.join(directory, pareto_file))
          # Merge the main and pareto datasets
          df_merged = pd.concat([df_main, df_pareto], ignore_index=True)
          # Optional: Save the merged dataset to a new CSV file in the subdirectory
          output_filename = f"{metric}_concat_{protein}.csv"
          output_path = os.path.join(directory, subdirectory, output_filename)
          os.makedirs(os.path.dirname(output_path), exist_ok=True)
          df_merged.to_csv(output_path, index=False)
          print(f"Merged dataset saved to {output_path}")


if __name__ == "__main__":
    # Specify the directory containing your CSV files
    directory = "./"
    merge_datasets(directory)

Merged dataset saved to ./concat/strain_enrichment_metrics_concat_S1PR1.csv
Merged dataset saved to ./concat/strain_roc_metrics_concat_S1PR1.csv
Merged dataset saved to ./concat/strain_log_aucs_concat_S1PR1.csv
Merged dataset saved to ./concat/strain_roc_metrics_concat_GPR40.csv
Merged dataset saved to ./concat/strain_enrichment_metrics_concat_GPR40.csv
Merged dataset saved to ./concat/strain_log_aucs_concat_GPR40.csv


In [2]:
import pandas as pd
import os

pd.set_option("display.max_rows", 200)

# Get all files in the 'concat' subdirectory of the current working directory
files = os.listdir("concat")

print(files)
# Step 1: Identify Unique Proteins

proteins = set(file.split("_")[3] for file in files if len(file.split("_")) >= 4)

# Initialize an empty DataFrame for the final combined data
combined_df = pd.DataFrame()

# Step 2 & 3: Process each protein
for protein in proteins:
    # Initialize empty lists to store DataFrames for each type
    enrichment_metrics_dfs = []
    log_aucs_dfs = []
    roc_metrics_dfs = []
    
    # Filter files for the current protein and process according to type
    for file in files:
        if protein in file:
            file_path = os.path.join("concat", file)  # Include the 'concat' directory in the file path
            if "enrichment_metrics" in file:
                enrichment_metrics_dfs.append(pd.read_csv(file_path))
            elif "log_aucs" in file:
                log_aucs_dfs.append(pd.read_csv(file_path))
            elif "roc_metrics" in file:
                roc_metrics_dfs.append(pd.read_csv(file_path))

    # Concatenate DataFrames of the same type
    enrichment_df = pd.concat(enrichment_metrics_dfs)
    log_aucs_df = pd.concat(log_aucs_dfs)
    roc_metrics_df = pd.concat(roc_metrics_dfs)

    # Step 4: Merge the three DataFrames for each protein
    merged_df = enrichment_df.merge(
        log_aucs_df, on=["Protein", "Strain Energy Cutoff"]
    ).merge(roc_metrics_df, on=["Protein", "Strain Energy Cutoff"])

    # Append to the final DataFrame
    combined_df = pd.concat([combined_df, merged_df], ignore_index=True)

# Resulting combined_df contains all data
display(combined_df)

# Save the final DataFrame to a CSV file
# combined_df.to_csv("combined_data.csv", index=False)

data = combined_df

['strain_enrichment_metrics_concat_S1PR1.csv', 'strain_log_aucs_concat_GPR40.csv', 'strain_roc_metrics_concat_S1PR1.csv', 'strain_roc_metrics_concat_GPR40.csv', 'strain_enrichment_metrics_concat_GPR40.csv', 'strain_log_aucs_concat_S1PR1.csv']


,Protein,Strain Energy Cutoff,EF1%,EF5%,deltaEF1%,deltaEF5%,Linear Log10 AUC (x10),Delta Linear Log10 AUC (x10),ROC_AUC,Actives,Total Count,deltaAUC
0,S1PR1,No Cutoff,3.271028,21.028037,0.000000,0.000000,3.133979,0.000000,0.800310,214,9107,0.000000
1,S1PR1,No Cutoff,3.271028,21.028037,0.000000,0.000000,3.133979,0.000000,0.800310,214,9107,0.000000
2,S1PR1,No Cutoff,3.271028,21.028037,0.000000,0.000000,3.133979,0.000000,0.800310,214,9107,0.000000
3,S1PR1,No Cutoff,3.271028,21.028037,0.000000,0.000000,3.133979,0.000000,0.800310,214,9107,0.000000
4,S1PR1,No Cutoff,3.271028,21.028037,0.000000,0.000000,3.133979,0.000000,0.800310,214,9107,0.000000
5,S1PR1,No Cutoff,3.271028,21.028037,0.000000,0.000000,3.133979,0.000000,0.800310,214,9107,0.000000
6,S1PR1,No Cutoff,3.271028,21.028037,0.000000,0.000000,3.133979,0.000000,0.800310,214,9107,0.000000
7,S1PR1,No Cutoff,3.271028,21.028037,0.000000,0.000000,3.133979,0.000000,0.800310,214,9107,0.000000
8,S1PR1,4,17.391304,34.782609,14.120276,13.754571,4.195943,1.061964,0.867996,23,1436,0.067686
9,S1PR1,4.5,18.181818,42.424242,14.910790,21.396205,4.498331,1.364352,0.866756,33,1859,0.066446


In [3]:
# Assuming df is your DataFrame
df = data.drop_duplicates(subset=["Protein", "Strain Energy Cutoff"])
display(df)

,Protein,Strain Energy Cutoff,EF1%,EF5%,deltaEF1%,deltaEF5%,Linear Log10 AUC (x10),Delta Linear Log10 AUC (x10),ROC_AUC,Actives,Total Count,deltaAUC
0,S1PR1,No Cutoff,3.271028,21.028037,0.000000,0.000000,3.133979,0.000000,0.800310,214,9107,0.000000
8,S1PR1,4,17.391304,34.782609,14.120276,13.754571,4.195943,1.061964,0.867996,23,1436,0.067686
9,S1PR1,4.5,18.181818,42.424242,14.910790,21.396205,4.498331,1.364352,0.866756,33,1859,0.066446
10,S1PR1,5.0,18.918919,43.243243,15.647891,22.215206,4.515744,1.381765,0.866626,37,2308,0.066316
11,S1PR1,5.5,20.000000,40.000000,16.728972,18.971963,4.394728,1.260749,0.864223,45,2773,0.063913
12,S1PR1,6.0,13.461538,34.615385,10.190510,13.587347,3.881042,0.747063,0.845620,52,3260,0.045310
13,S1PR1,7.0,11.594203,27.536232,8.323175,6.508195,3.524028,0.390049,0.826156,69,4252,0.025846
14,S1PR1,7.5,10.126582,29.113924,6.855554,8.085887,3.560316,0.426337,0.829510,79,4766,0.029199
15,S1PR1,8.0,8.888889,26.666667,5.617861,5.638629,3.482260,0.348281,0.828585,90,5282,0.028274
16,S1PR1,Top 10 Pareto Ranks,0.000000,0.000000,-3.271028,-21.028037,2.173813,-0.960166,0.745252,20,357,-0.055058
